# BTSnoop Log Sniffer

This notebook demonstrates the process of parsing bluetooth packet data from android hci logs.

In [46]:
# !pip install fpdf
# !pip install kaleido
# !pip install dataframe_image

In [47]:
import sys
import json
import time
import requests
from constants import *

import fpdf
from fpdf import FPDF
import pandas as pd
import numpy as np
from pprint import pprint
from decimal import Decimal
import dataframe_image as dfi
from datetime import datetime

# import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots


pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)


In [48]:
# filepath = sys.argv[1]

# with open(filepath, 'r') as file:
with open('./output1.json', 'r') as file:
    data = [x["_source"]["layers"] for x in json.load(file)]

[source](https://github.com/nccgroup/BLE-Replay/blob/master/btsnoop/btsnoop/bt/hci_evt.py)

The HCI LE Meta Event is used to encapsulate all LE Controller specific events.
The Event Code of all LE Meta Events shall be 0x3E. The Subevent_Code is
the first octet of the event parameters. The Subevent_Code shall be set to one
of the valid Subevent_Codes from an LE specific event


HCI inherently cannot differentiate between packet types. Hence a common physical interface is used with the indicators that are sent right before the packet is sent. These indicators are as follows

| HCI Packet Type | HCI Packet Indicator |
| --------------- | -------------------- |
| HCI Command Packet | 0x01 |
| HCI ACL Data Packet | 0x02 |
| HCI Synchronous Data Packet | 0x03 |
| HCI Event Packet | 0x04 |
| HCI ISO Data Packet | 0x05 |

LE Meta events encapsulate all LE Controller events and have a code of 0x03

protocols = {
    "bluetooth:hci_h4:bthci_acl:btl2cap": 'ACL-L2CAP Event',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btatt": 'ACL-L2CAP - ATT (Attribute) Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btavctp:btavrcp": 'ACL-L2CAP - AVCTP/AVR Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btavdtp": 'ACL-L2CAP - AVDTP Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm": 'ACL-L2CAP - RF Communication protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm:bthfp": 'ACL-L2CAP - Hands Free Protocol',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btrfcomm:data": 'ACL-L2CAP - Data Transfer',
    "bluetooth:hci_h4:bthci_acl:btl2cap:btsdp": 'ACL-L2CAP - Service Discovery Protocol (SDP)',
    "bluetooth:hci_h4:bthci_cmd": 'HCI Command',
    "bluetooth:hci_h4:bthci_cmd:btcommon": 'Bluetooth Advertising Event',
    "bluetooth:hci_h4:bthci_cmd:bthci_vendor.broadcom": 'Vendor-Specific Broadcom Packet',
    "bluetooth:hci_h4:bthci_evt": 'HCI Event',
    "bluetooth:hci_h4:bthci_evt:btcommon": 'Bluetooth Advertising Event',
    "bluetooth:hci_h4:bthci_evt:bthci_vendor.broadcom": 'Vendor-Specific Broadcom Packet'
    }

In [49]:
def format_packet_data(data):
    data_input = data
    for packet in data_input:
        packet["Sequence Number"] = packet["frame.number"][0] if "frame.number" in packet else None
        packet["Epoch Timestamp"] = packet["frame.time_epoch"][0] if "frame.time_epoch" in packet else None
        packet["Timestamp"] = datetime.fromtimestamp(int(Decimal(packet["frame.time_epoch"][0]))) if "frame.time_epoch" in packet else None
        packet["Packet Length"] = packet["frame.len"][0] if "frame.len" in packet else None
        packet["LE Protocol"] = PROTOCOL_TYPES[packet["frame.protocols"][0]] if "frame.protocols" in packet else None
        packet["LE Event Code"] = packet["hci_h4.type"][0] if "hci_h4.type" in packet else None
        packet["LE Event Type"] = HCI_LE_EVENT[packet["hci_h4.type"][0]] if "hci_h4.type" in packet else None
        packet["Direction"] = EVENT_DIRECTION[packet["hci_h4.direction"][0]] if "hci_h4.direction" in packet else None
        packet["HCI Command"] = packet["bthci_cmd"][0] if "bthci_cmd" in packet else None
        packet["HCI Event Code"] = packet["bthci_evt.code"][0] if "bthci_evt.code" in packet else None
        packet["HCI Event"] = packet["bthci_evt"][0] if "bthci_evt" in packet else None
        packet["HCI Event Command Packet Count"] = packet["bthci_evt.num_command_packets"][0] if "bthci_evt.num_command_packets" in packet else None
        packet["HCI Event Status"] = packet["bthci_evt.status"][0] if "bthci_evt.status" in packet else None
        packet["HCI Event Command in Frame"] = packet["bthci_evt.command_in_frame"][0] if "bthci_evt.command_in_frame" in packet else None
        packet["HCI ACL chandle"] = packet["bthci_acl.chandle"][0] if "bthci_acl.chandle" in packet else None
        packet["HCI ACL Length"] = packet["bthci_acl.length"][0] if "bthci_acl.length" in packet else None
        packet["Source Device MAC"] = packet["bthci_acl.src.bd_addr"][0] if "bthci_acl.src.bd_addr" in packet else None
        packet["Source Device Name"] = packet["bthci_cmd.device_name"][0] if "bthci_cmd.device_name" in packet else (packet["bthci_acl.src.name"][0] if "bthci_acl.src.name" in packet else "Unknown Device")
        packet["Destination Device MAC"] = packet["bthci_acl.dst.bd_addr"][0] if "bthci_acl.dst.bd_addr" in packet else None
        packet["Destination Device Name"] = packet["bthci_acl.dst.name"][0] if "bthci_acl.dst.name" in packet else None
        packet["L2CAP CID"] = packet["btl2cap.cid"][0] if "btl2cap.cid" in packet else None
        packet["L2CAP Length"] = packet["btl2cap.length"][0] if "btl2cap.length" in packet else None


        if packet["Source Device Name"] == "":
            packet["Source Device Name"] = f"Unknown Device-{packet['Source Device MAC'][:8]}" if packet["Source Device MAC"] else "Unknown Device"

        if packet["Destination Device Name"] == "":
            packet["Destination Device Name"] = f"Unknown Device-{packet['Destination Device MAC'][:8]}" if packet["Destination Device MAC"] else "Unknown Device"


        if packet["L2CAP CID"] is not None:
            if packet["L2CAP CID"] in L2CAP_CID_VALUES:
                packet["L2CAP Type"] = L2CAP_CID_VALUES[packet["L2CAP CID"]]
            else:
                packet["L2CAP Type"] = 'Dynamically Allocated'

        for key in PACKET_KEYS:
            if key in packet:
                del packet[key]

        keys_to_delete = [key for key, value in packet.items() if value is None]
        for key in keys_to_delete:
            del packet[key]
    
    return pd.DataFrame(data)

In [50]:
df_data = format_packet_data(data)

In [51]:
df_data.head()

,frame.time_utc,Sequence Number,Epoch Timestamp,Timestamp,Packet Length,LE Protocol,LE Event Code,LE Event Type,Direction,HCI Command,Source Device Name,HCI Event Code,HCI Event,HCI Event Command Packet Count,HCI Event Status,HCI Event Command in Frame,HCI ACL chandle,HCI ACL Length,Source Device MAC,Destination Device MAC,Destination Device Name,L2CAP CID,L2CAP Length,L2CAP Type
0,"[Oct 3, 2024 03:12:15.575141000 UTC]",1,1727925135.575141000,2024-10-02 23:12:15,4,HCI Command,0x01,Command (HCI_CMD),Host > Controller,Bluetooth HCI Command - Reset,Unknown Device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"[Oct 3, 2024 03:12:15.580693000 UTC]",2,1727925135.580693000,2024-10-02 23:12:15,7,HCI Command,0x04,Event (HCI_EVT),Controller > Host,NaN,Unknown Device,0x0e,Bluetooth HCI Event - Command Complete,1,0x00,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"[Oct 3, 2024 03:12:15.580795000 UTC]",3,1727925135.580795000,2024-10-02 23:12:15,12,HCI Command,0x01,Command (HCI_CMD),Host > Controller,Bluetooth HCI Command - Set Event Mask,Unknown Device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"[Oct 3, 2024 03:12:15.581154000 UTC]",4,1727925135.581154000,2024-10-02 23:12:15,7,HCI Command,0x04,Event (HCI_EVT),Controller > Host,NaN,Unknown Device,0x0e,Bluetooth HCI Event - Command Complete,1,0x00,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"[Oct 3, 2024 03:12:15.581234000 UTC]",5,1727925135.581234000,2024-10-02 23:12:15,6,HCI Command,0x01,Command (HCI_CMD),Host > Controller,Bluetooth HCI Command - Write LE Host Supported,Unknown Device,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
def generate_device_piecharts(df, names, title, export_filename):
    fig = px.pie(df, names=names, title=title, hole=0.3, width=750, height=400)
    fig.update_layout(title_font_size=24)
    fig.update_traces(textinfo='percent', texttemplate='%{percent:.2%}', textfont_size=18)
    fig.update_layout(legend=dict(x=0.85, y=1.1))
    fig.update_layout(margin=dict(t=50, b=30, l=0, r=220))
    fig.write_image(f'./generated_images/{export_filename}', scale=2)
    # fig.show()

In [53]:
source_mac_info = df_data['Source Device Name'] + "\n(" + df_data['Source Device MAC'] + ")"
source_mac_info = source_mac_info.dropna()
df_sources = pd.DataFrame(source_mac_info, columns=['Source Devices'])

generate_device_piecharts(df_sources, 'Source Devices', 'Distribution of Source Devices', 'source_devices_distribution.png')

In [54]:
destination_device_info = df_data['Destination Device Name'] + "\n(" + df_data['Destination Device MAC'] + ")"
destination_device_info = destination_device_info.dropna()
df_destinations = pd.DataFrame(destination_device_info, columns=['Destination Devices'])

generate_device_piecharts(df_destinations, 'Destination Devices', 'Distribution of Destination Devices', 'destination_devices_distribution.png')

In [55]:
def generate_hci_cmdevt_plot(df_hci_cmd, df_hci_event, export_filename):
    fig = make_subplots(specs=[[{"secondary_y": True}]])

    fig.add_trace(
        go.Scatter(
            x=df_hci_cmd['Timestamp'],
            y=df_hci_cmd['Sequence Number'],
            mode='lines+markers',
            name='HCI Commands',
            line=dict(color='red')),
            secondary_y=False
        )

    fig.add_trace(
        go.Scatter(
            x=df_hci_event['Timestamp'],
            y=df_hci_event['Sequence Number'],
            mode='lines+markers',
            name='HCI Events',
            line=dict(color='blue')),
            secondary_y=True
        )

    fig.update_yaxes(title_text="Sequence Number (HCI Commands)", dtick=200, tickmode='linear', secondary_y=False)
    fig.update_yaxes(title_text="Sequence Number (HCI Events)", dtick=200, tickmode='linear', secondary_y=True)

    fig.update_layout(title='HCI Events and Commands over Time',
                      title_font_size=24,
                      xaxis_title='Timestamp (UTC)',
                      xaxis=dict(tickangle=45),
                      legend=dict(x=0.01, y=0.99),
                      template='plotly_white',
                      width=900,
                      height=600)
    fig.write_image(f'./generated_images/{export_filename}', scale=2)
    # fig.show()

In [56]:
df_hci_cmd = df_data[df_data['LE Event Type'] == 'Command (HCI_CMD)'].dropna(axis=1, how='all')
df_hci_event = df_data[df_data['LE Event Type'] == 'Event (HCI_EVT)'].dropna(axis=1, how='all')
df_hci_event = df_hci_event[df_hci_event['HCI Event Command in Frame'].isin(df_hci_cmd['Sequence Number'])]

generate_hci_cmdevt_plot(df_hci_cmd, df_hci_event, export_filename="hci_cmd_event_communication.png")

In [57]:
mac_addresses = pd.DataFrame({
    'MAC Address': np.unique(
        np.concatenate((df_data['Source Device MAC'].dropna().unique(), df_data['Destination Device MAC'].dropna().unique()))
        )
})

In [58]:
def get_mac_info(mac_address):
    headers = {
        'Content-Type': 'application/json',
        'Accept': 'application/json'
    }
    url = VENDOR_LOOKUP_API + mac_address
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()[0]
    else:
        return None

def generate_mac_info_dataframe(unique_mac_list):
    unique_mac_info_list = []
    for mac in unique_mac_list:
        info = get_mac_info(mac)
        if info:
            unique_mac_info_list.append(info)
    mac_info_df = pd.DataFrame(unique_mac_info_list)[['company', 'addressL1', 'addressL2', 'addressL3', 'country']]
    return mac_info_df

In [59]:
mac_vendor_info = generate_mac_info_dataframe(mac_addresses['MAC Address'].tolist())
mac_vendor_info.head()

,company,addressL1,addressL2,addressL3,country
0,"GooWi Wireless Technology Co., Limited","RM1601,Crative BuildingII East Tianan",City Futian Shenzhen Guangdong 518000,,CN
1,"Xi'an Yipu Telecom Technology Co.,Ltd.","Floor 5, Block C, Huanpu Industrial Park, 211 Tiangu 8th Road",Xi 'an Shaanxi 710076,,CN
2,"Google, Inc.",1600 Amphitheatre Parkway,Mountain View CA 94043,,US


In [60]:
df_acl_le_events = df_data[(df_data['LE Event Type'] == 'ACL (HCI_ACL)') & (df_data['LE Protocol'].str.contains('ACL'))]
df_acl_le_events = df_acl_le_events.dropna(axis=1, how='all')


In [61]:
df_acl_le_events.head()

,frame.time_utc,Sequence Number,Epoch Timestamp,Timestamp,Packet Length,LE Protocol,LE Event Code,LE Event Type,Direction,Source Device Name,HCI ACL chandle,HCI ACL Length,Source Device MAC,Destination Device MAC,Destination Device Name,L2CAP CID,L2CAP Length,L2CAP Type
144,"[Oct 3, 2024 03:12:16.373296000 UTC]",145,1727925136.373296000,2024-10-02 23:12:16,12,ACL - Attribute Protocol,0x02,ACL (HCI_ACL),Controller > Host,Unknown Device-d0:62:2c,0x0040,7,d0:62:2c:59:12:92,d4:3a:2c:a9:32:4f,Pixel 7 Pro,0x0004,3,Attribute Protocol (ATT)
145,"[Oct 3, 2024 03:12:16.374463000 UTC]",146,1727925136.374463000,2024-10-02 23:12:16,12,ACL - Attribute Protocol,0x02,ACL (HCI_ACL),Host > Controller,Pixel 7 Pro,0x0040,7,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,3,Attribute Protocol (ATT)
153,"[Oct 3, 2024 03:12:16.420924000 UTC]",154,1727925136.420924000,2024-10-02 23:12:16,16,ACL - Attribute Protocol,0x02,ACL (HCI_ACL),Host > Controller,Pixel 7 Pro,0x0040,11,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,7,Attribute Protocol (ATT)
208,"[Oct 3, 2024 03:12:16.508358000 UTC]",209,1727925136.508358000,2024-10-02 23:12:16,14,ACL - Attribute Protocol,0x02,ACL (HCI_ACL),Controller > Host,Unknown Device-d0:62:2c,0x0040,9,d0:62:2c:59:12:92,d4:3a:2c:a9:32:4f,Pixel 7 Pro,0x0004,5,Attribute Protocol (ATT)
210,"[Oct 3, 2024 03:12:16.509110000 UTC]",211,1727925136.509110000,2024-10-02 23:12:16,16,ACL - Attribute Protocol,0x02,ACL (HCI_ACL),Host > Controller,Pixel 7 Pro,0x0040,11,d4:3a:2c:a9:32:4f,d0:62:2c:59:12:92,Unknown Device-d0:62:2c,0x0004,7,Attribute Protocol (ATT)


In [62]:
df_data['LE Protocol'].unique()

array(['HCI Command', 'HCI Command (Vendor Specific)',
       'HCI Event (Vendor Specific)', 'HCI Command (btcommon)',
       'ACL - Attribute Protocol', 'HCI Event (btcommon)',
       'ACL - L2CAP Protocol', 'ACL - Service Discovery Protocol',
       'ACL - Audio/Video Distribution Transport Protocol',
       'ACL - Radio Frequency Communication',
       'ACL - RFComm Hands Free Profile', 'ACL - RFComm Data Transfer',
       'ACL - Audio/Video Remote Control Profile'], dtype=object)

In [63]:
# Summary statistics
total_hci_cmds = len(df_hci_cmd)
total_hci_events = len(df_hci_event)

total_acl_events = len(df_acl_le_events)
acl_protocols = df_acl_le_events['LE Protocol'].nunique()
packet_size_distribution = df_acl_le_events['HCI ACL Length'].describe() if 'HCI ACL Length' in df_acl_le_events.columns else None


summary_stats = {
    'Number of HCI Commands': total_hci_cmds,
    'Number of HCI Events': total_hci_events,
    'Number of ACL Events': total_acl_events,
    'ACL Protocols': acl_protocols,
    'Packet Length Distribution': packet_size_distribution
}


print("Summary Statistics:")
print(f"Number of ACL Events: {total_acl_events}")
print(f"ACL Protocols: {acl_protocols}")
print(f"Packet Length Distribution:\n{packet_size_distribution}")

Summary Statistics:
Number of ACL Events: 3355
ACL Protocols: 8
Packet Length Distribution:
count     3355
unique     121
top        251
freq       494
Name: HCI ACL Length, dtype: object


In [64]:
def create_title(title, pdf):
    
    # PDF Title
    pdf.set_font('Helvetica', 'b', 24)  
    pdf.ln(40)
    pdf.write(5, title)
    pdf.ln(10)
    
    # Report Date
    pdf.set_font('Helvetica', '', 16)
    pdf.set_text_color(r=128,g=128,b=128)
    today = time.strftime("%d/%m/%Y")
    pdf.write(4, f'{today}')
    
    # Add line break
    pdf.ln(10)

def create_heading(title, pdf):
    pdf.set_font('Helvetica', 'b', 18)  
    pdf.ln(40)
    pdf.write(5, title)
    pdf.ln(10)


def write_to_pdf(pdf, words):
    # Set text colour, font size, and font type
    pdf.set_text_color(r=0,g=0,b=0)
    pdf.set_font('Helvetica', '', 12)
    pdf.write(5, words)

In [65]:
class PDF(FPDF):
    def footer(self):
        self.set_y(-15)
        self.set_font('Helvetica', 'I', 8)
        self.set_text_color(128)
        self.cell(0, 10, 'Page ' + str(self.page_no()), 0, 0, 'C')

In [ ]:
TITLE = "Bluetooth Packet Capture Report"
WIDTH = 210
HEIGHT = 297

first_timestamp = df_data['Timestamp'].min()
last_timestamp = df_data['Timestamp'].max()

pdf = PDF()

pdf.add_page()

create_title(TITLE, pdf)

write_to_pdf(pdf, "This document summarizes some of the main Bluetooth communication packets")

pdf.ln(10)

write_to_pdf(pdf, "Key Points:")
pdf.ln(10)
pdf.set_left_margin(30)
write_to_pdf("- Number of ACL Events: " + str(summary_stats['Number of ACL Events']))
pdf.ln(5)
pdf.write(5, "- ACL Protocols: " + str(summary_stats['ACL Protocols']))
pdf.ln(5)
pdf.write(5, "- Packet Length Distribution: " + str(summary_stats['Packet Length Distribution']))


AttributeError: 'int' object has no attribute 'set_text_color'

The Bluetooth Asynchronous Connection-oriented Logical transport (ACL) is a method of transfer of asynchronous (data that is not in real time) data over a bluetooth connection.

Looking at some of the profiles under ACL we see the following:

- *AVDTP*: This is the Audio/Video Distribution Transport Protocol (AVDTP) signifies the streaming of music to a bluetooth speaker device such as a set of headsets over the L2CAP protocol.
- *HFP*: The hands free profile (HFP) can be seen as a mode of remote control between the phone and bluetooth device.
- *AVRCP*: The Audio/Video Remote Control Profile is used along with the A2DP (Advanced Audio Distribution Profile) profile to allow a single remote device to control a bluetooth device
- *SDP*: The Service Discovert Protocol is a manner of communication to discover the available bluetooth devices nearby and their types.

[Reference](https://datatracker.ietf.org/doc/html/rfc1761)

Android bluetooth logs come in the **Snoop Version 1 Packet Capture File Format** which is similar to the second version developed by Sun Microsystems in 1995. When we capture logs of this format, we obtain arrays of octets (8 bit packets of information), with each array item corresponding to a packet record.
